In [ ]:
# Standard library imports
import os
import random
import warnings

# Third-party library imports for data manipulation
import numpy as np
import pandas as pd

# Visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns
import ptitprince as pt
import umap

# Machine Learning and statistical testing libraries
from scipy.stats import mannwhitneyu, f_oneway
from sklearn.ensemble import RandomForestClassifier
from sklearn.exceptions import FitFailedWarning
from sklearn.feature_selection import VarianceThreshold
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler, RobustScaler

# Survival analysis libraries
import sksurv
from sksurv.compare import compare_survival
from sksurv.linear_model import CoxPHSurvivalAnalysis, CoxnetSurvivalAnalysis
from sksurv.metrics import concordance_index_censored, concordance_index_ipcw, integrated_brier_score
from sksurv.nonparametric import kaplan_meier_estimator
from sksurv.util import Surv

# Feature selection libraries
from boruta import BorutaPy
from lifelines import CoxPHFitter
# Lifelines for additional survival analysis
from lifelines.statistics import logrank_test, multivariate_logrank_test
from tqdm import tqdm
from lifelines import KaplanMeierFitter
from lifelines.plotting import add_at_risk_counts
import seaborn as sns
import umap
from dcurves import dca, plot_graphs

### Helper code

In [ ]:
def calculate_survival_benefit(data_df1, data_df2, time_points=[1,3, 5,7, 10]):
    # Initialize Kaplan-Meier fitters for each group
    kmf_hr = KaplanMeierFitter()  # High risk group
    kmf_lr = KaplanMeierFitter()  # Low risk group

    # Fit data for high risk group
    kmf_hr.fit(data_df1['time'], event_observed=data_df1['event'])

    # Fit data for low risk group
    kmf_lr.fit(data_df2['time'], event_observed=data_df2['event'])

    # Calculate survival probabilities at specified time points
    survival_hr = kmf_hr.survival_function_.reindex(time_points, method='nearest')
    survival_lr = kmf_lr.survival_function_.reindex(time_points, method='nearest')

    # Calculate differences in survival probabilities and format them
    survival_benefits = {}
    for t in time_points:
        prob_hr = kmf_hr.predict(t)
        prob_lr = kmf_lr.predict(t)
        survival_benefit = (prob_lr - prob_hr) * 100  # in percentage points
        survival_benefits[t] = survival_benefit

    return survival_benefits


def plot_km_same_risk_group(data_df1, data_df2, data_stats, figure_save_path, title_str):
    
    # Create a colormap
    cmap = plt.cm.get_cmap('Reds')
    # Choose a shade of red
    hr_shade = cmap(0.75)

    cmap = plt.cm.get_cmap('Blues')
    # Choose a shade of red
    lr_shade = cmap(0.75)
    
    fig, ax = plt.subplots(1, 1, figsize=(15, 12))

    data_high_risk = data_df1
    data_low_risk = data_df2

    kmf_hr = KaplanMeierFitter()
    kmf_hr.fit(data_high_risk['time'], event_observed=data_high_risk['event'], label='RT+ADT')
    kmf_hr.plot_survival_function(ax=ax, color='#f8766d', lw=2, show_censors=True)

    kmf_lr = KaplanMeierFitter()
    kmf_lr.fit(data_low_risk['time'], event_observed=data_low_risk['event'], label='RT+ADT+CT')
    kmf_lr.plot_survival_function(ax=ax, color='#03bfc4', lw=2, show_censors=True, )
    
    
    print("Low risk median survival time: ",kmf_lr.median_survival_time_)
    print("High risk median survival time: ",kmf_hr.median_survival_time_)
    max_median_survival_time = max(kmf_hr.median_survival_time_ , kmf_lr.median_survival_time_)
    if max_median_survival_time == np.inf:
        plt.axhline(y=0.5, color='black', linestyle='--', lw=1)
    else:
        plt.plot([0, max_median_survival_time], [0.5, 0.5], color='black', linestyle='--', lw=1)
    # Vertical lines up to y=0.5, using plot for precise control
    plt.plot([kmf_hr.median_survival_time_, kmf_hr.median_survival_time_], [0, 0.5], color='black', linestyle='--', lw=1)
    plt.plot([kmf_lr.median_survival_time_, kmf_lr.median_survival_time_], [0, 0.5], color='black', linestyle='--', lw=1)
    ax.set_title(title_str, fontsize=26)

    yticks = [np.round(x,1) for x in ax.get_yticks()]
    ax.set_yticklabels(yticks, fontsize=20)
    ax.set_xticklabels(ax.get_xticks().astype(int), fontsize=20)


    ax.set_xlabel('Time (Years)', fontsize=28)
    ax.set_ylabel('Overall Survival Probability', fontsize=28)
    data_p, data_hr, data_ci_lower, data_ci_upper = data_stats
    format_p = lambda p: f"{p:.1e}" if p < 0.001 else f"{p:.4f}"

# Updated string formatting
    data_stats_text = f'p: {format_p(data_p)}\nHR: {data_hr:.2f} [95% CI: {data_ci_lower:.2f} - {data_ci_upper:.2f}]'
    #data_stats_text = f'p: {data_p:.1e if data_p < 0.001 else data_p:.4f}\nHR: {data_hr:.2f} [95% CI: {data_ci_lower:.2f} - {data_ci_upper:.2f}]'
    ax.text(0.03, 0.1, data_stats_text, transform=ax.transAxes, fontsize=24, verticalalignment='bottom')

    # Add the risk table at the bottom of the KM plot on ax[1] (the bottom subplot)
    sns.despine()
    add_at_risk_counts(kmf_hr, kmf_lr, ax=ax, fontsize=20)
    ax.legend(fontsize=24)
    plt.tight_layout()
    
    #fig.savefig(figure_save_path, bbox_inches='tight')
    plt.show()

def plot_km_curve_lifelines(data_df, data_stats, figure_save_path, title_str):
    
    # Create a colormap
    cmap = plt.cm.get_cmap('Reds')
    # Choose a shade of red
    hr_shade = cmap(0.75)

    cmap = plt.cm.get_cmap('Blues')
    # Choose a shade of red
    lr_shade = cmap(0.75)
    
    fig, ax = plt.subplots(1, 1, figsize=(15, 12))

    data_high_risk = data_df[data_df['risk_group']==1]
    data_low_risk = data_df[data_df['risk_group']==0]

    kmf_hr = KaplanMeierFitter()
    kmf_hr.fit(data_high_risk['time'], event_observed=data_high_risk['event'], label='High Risk')
    kmf_hr.plot_survival_function(ax=ax, color='#f8766d', lw=2, show_censors=True)

    kmf_lr = KaplanMeierFitter()
    kmf_lr.fit(data_low_risk['time'], event_observed=data_low_risk['event'], label='Low Risk')
    kmf_lr.plot_survival_function(ax=ax, color='#03bfc4', lw=2, show_censors=True, )
    
    
    print("Low risk median survival time: ",kmf_lr.median_survival_time_)
    print("High risk median survival time: ",kmf_hr.median_survival_time_)
    max_median_survival_time = max(kmf_hr.median_survival_time_ , kmf_lr.median_survival_time_)
    if max_median_survival_time == np.inf:
        plt.axhline(y=0.5, color='black', linestyle='--', lw=1)
    else:
        plt.plot([0, max_median_survival_time], [0.5, 0.5], color='black', linestyle='--', lw=1)
    # Vertical lines up to y=0.5, using plot for precise control
    plt.plot([kmf_hr.median_survival_time_, kmf_hr.median_survival_time_], [0, 0.5], color='black', linestyle='--', lw=1)
    plt.plot([kmf_lr.median_survival_time_, kmf_lr.median_survival_time_], [0, 0.5], color='black', linestyle='--', lw=1)
    ax.set_title(title_str, fontsize=26)

    yticks = [np.round(x,1) for x in ax.get_yticks()]
    ax.set_yticklabels(yticks, fontsize=20)
    ax.set_xticklabels(ax.get_xticks().astype(int), fontsize=20)


    ax.set_xlabel('Time (Years)', fontsize=28)
    ax.set_ylabel('Overall Survival Probability', fontsize=28)
    data_p, data_hr, data_ci_lower, data_ci_upper = data_stats
    format_p = lambda p: f"{p:.1e}" if p < 0.001 else f"{p:.4f}"

# Updated string formatting
    data_stats_text = f'p: {format_p(data_p)}\nHR: {data_hr:.2f} [95% CI: {data_ci_lower:.2f} - {data_ci_upper:.2f}]'
    #data_stats_text = f'p: {data_p:.1e if data_p < 0.001 else data_p:.4f}\nHR: {data_hr:.2f} [95% CI: {data_ci_lower:.2f} - {data_ci_upper:.2f}]'
    ax.text(0.03, 0.1, data_stats_text, transform=ax.transAxes, fontsize=24, verticalalignment='bottom')

    # Add the risk table at the bottom of the KM plot on ax[1] (the bottom subplot)
    sns.despine()
    add_at_risk_counts(kmf_hr, kmf_lr, ax=ax, fontsize=20)
    ax.legend(fontsize=24)
    plt.tight_layout()
    
    #fig.savefig(figure_save_path, bbox_inches='tight')
    plt.show()


def remove_correlated_features(df, threshold=0.95):
    # Create correlation matrix
    corr_matrix = pd.DataFrame(np.corrcoef(df.values, rowvar=False), columns=df.columns).abs() 

    # Select upper triangle of correlation matrix
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

    # Find features with correlation greater than 0.95
    to_drop = [column for column in upper.columns if any(upper[column] > threshold)]
    # len(to_drop)

    # Drop features 
    corr_removed_df = df.drop(to_drop, axis=1)
    
    return corr_removed_df


def plot_coefficients(coefs, n_highlight):
    _, ax = plt.subplots(figsize=(9, 6))
    n_features = coefs.shape[0]
    alphas = coefs.columns
    for row in coefs.itertuples():
        ax.semilogx(alphas, row[1:], ".-", label=row.Index)

    alpha_min = alphas.min()
    top_coefs = coefs.loc[:, alpha_min].map(abs).sort_values().tail(n_highlight)
    for name in top_coefs.index:
        coef = coefs.loc[name, alpha_min]
        plt.text(alpha_min, coef, name + "   ", horizontalalignment="right", verticalalignment="center")

    ax.yaxis.set_label_position("right")
    ax.yaxis.tick_right()
    ax.grid(True)
    ax.set_xlabel("alpha")
    ax.set_ylabel("coefficient")
    
def boruta_selected_features(feature_df, y):
    # define Boruta feature selection method
    # ipdb.set_trace()
    rf = RandomForestClassifier(n_jobs=-1, class_weight='balanced', max_depth=5)

    feat_selector = BorutaPy(rf, n_estimators='auto', perc=95, alpha=0.05, two_step=False,verbose=0, random_state=42)

    feat_selector.fit(feature_df.values, y)

    # check selected features
    boruta_selected_features = feature_df.columns[feat_selector.support_  | feat_selector.support_weak_].to_list()
    boruta_selected_features_df = feature_df[boruta_selected_features]
    
    return boruta_selected_features_df

def get_discriminative_features(view_features):

    view_features_trim1 = view_features.drop(['patient_id', 'event', 'time'], axis=1) #'view', , 'img_id'
    y = view_features['event']
    if scaler_type == "minmax":
        scaler = MinMaxScaler() #StandardScaler() #RobustScaler() #
    else:
        scaler = RobustScaler(unit_variance=True)
    view_features_scaled = pd.DataFrame(scaler.fit_transform(view_features_trim1.values), columns=view_features_trim1.columns)

    view_features_trim2 = remove_correlated_features(view_features_scaled, threshold=0.85)


    # drop columns with zero variance using sklearn's VarianceThreshold
    sel = VarianceThreshold(threshold=0.01)
    sel.fit(view_features_trim2)
    view_features_trimmed = view_features_trim2[view_features_trim2.columns[sel.get_support(indices=True)]]
    rf = RandomForestClassifier(n_jobs=-1, class_weight='balanced', max_depth=5)

    feat_selector = BorutaPy(rf, n_estimators='auto', perc=98, alpha=0.05, two_step=False,verbose=0, random_state=42)

    feat_selector.fit(view_features_trimmed.values, y)


    # check selected features
    boruta_selected_features = view_features_trimmed.columns[feat_selector.support_].to_list() # | feat_selector.support_weak_

    disc_features = view_features_trimmed[boruta_selected_features]
    disc_features = pd.DataFrame(scaler.fit_transform(disc_features.values), columns=disc_features.columns)

    return disc_features#, sig_pvals

def plot_raincloudplots(df, stable_features, events_df, save_path):
    rc_plot_df = df.copy()
    rc_plot_df['event'] = events_df
    for feature in stable_features:
        fig = plt.figure(figsize=(5,4))
        # Perform wilcoxon test to check if the feature is significantly different between the two groups
        
        stat, pvalue = f_oneway(rc_plot_df[events_df==1][feature].values, rc_plot_df[events_df==0][feature].values)
        
        pt.RainCloud(x='event', y=feature, data=rc_plot_df)
        plt.title('p_val: '+str(pvalue))
        plt.show()
        fig.savefig(save_path+'_'+feature+'.png', dpi=300, bbox_inches='tight')
        
def get_hr_and_pval(threshold, val_risk_scores, y_val_survlabel):

    # Calculate Kaplan-Meier estimator for different risk groups
    risk_groups = threshold #np.mean(val_risk_scores)#np.percentile(test_risk_scores, 50)
    risk_group_labels = np.array([1 if x > risk_groups else 0 for x in val_risk_scores])#np.digitize(test_risk_scores, risk_groups)
    survival_probs = []
    survival_times = []

    for group_label in np.unique(risk_group_labels):
        group_indices = np.where(risk_group_labels == group_label)
        group_time, group_survival_prob = kaplan_meier_estimator(events[group_indices], times[group_indices])
        survival_probs.append(group_survival_prob)
        survival_times.append(group_time)


    tstat, pval, stats0, stats1 = compare_survival(y_val_survlabel, risk_group_labels, return_stats=True)

    return tstat, pval

def get_cph_model_stats(cph_model_results, c_index):
    
    cph_p_value = cph_model_results['p'].values[0]
    cph_HR = cph_model_results['exp(coef)'].values[0]
    cph_CI_lower = cph_model_results['exp(coef) lower 95%'].values[0]
    cph_CI_upper = cph_model_results['exp(coef) upper 95%'].values[0]
    
    
    return (cph_p_value, cph_HR, cph_CI_lower, cph_CI_upper, c_index)

def get_multivariate_coxph_results(risk_df, drop_cols=['patient_id']):
    
    cph = CoxPHFitter(l1_ratio=0.7)
    risk_vars = risk_df.drop(columns=drop_cols)
    cph.fit(risk_vars, duration_col='time', event_col='event', show_progress=False)
    results = cph.summary
    
    return results, cph

def get_multivariate_coxph_results_stratified(risk_df, drop_cols=['patient_id'], strata_cols=['gleason']):
    # Initialize the Cox proportional hazards model with regularization
    cph = CoxPHFitter(l1_ratio=0.7)
    
    # Drop specified columns from the DataFrame
    risk_vars = risk_df.drop(columns=drop_cols)
    
    # Fit the model with stratification on the specified columns
    cph.fit(risk_vars, duration_col='time', event_col='event', strata=strata_cols, show_progress=False)
    
    # Get the summary of the model's results
    results = cph.summary
    
    return results, cph

def plot_n_save_adjusted_hazard(cph_training_adjusted_risk, cph_holdout_adjusted_risk,training_df, holdout_df, save_path, title='Adjusted Risk model'):
    
    fig, ax = plt.subplots(1, 2, figsize=(25,10))

    cph_training_adjusted_risk.plot(hazard_ratios=True, ax = ax[0])
    ax[0].set_title('Training Patients', fontsize=23)
    ax[0].set_ylabel('Covariates', fontsize=20)
    ax[0].set_xlabel('Hazard Ratio (95% CI)', fontsize=18)
    ax[0].set_xticklabels(ax[0].get_xticklabels(), fontsize=18)
    ax[0].set_yticklabels(ax[0].get_yticklabels(), fontsize=18)
    
    # increase the space between the two subplots
    fig.subplots_adjust(wspace=0.5)
    
    cph_holdout_adjusted_risk.plot(hazard_ratios=True, ax = ax[1])
    ax[1].set_title('Holdout Patients'.format(len(holdout_df)), fontsize=20)

    
    ax[1].set_xlabel('Hazard Ratio (95% CI)', fontsize=18)
    ax[1].set_xticklabels(ax[1].get_xticklabels(), fontsize=18)
    ax[1].set_yticklabels(ax[1].get_yticklabels(), fontsize=18)

    plt.suptitle(title, fontsize=25, y=0.98)
    #fig.savefig(save_path)
    plt.show()

warnings.simplefilter("ignore", UserWarning)
warnings.simplefilter("ignore", FitFailedWarning)
warnings.simplefilter("ignore", RuntimeWarning)

### Survival Analysis

In [ ]:
features = pd.read_csv(r"combined_fixed_final.csv")

In [ ]:
features.rename(columns={'survival_years': 'time', 'survival': 'event'}, inplace=True)
features = features.drop(columns=['disease_free_survival', 'disease_free_survival_years', 'any_distant_mets', 'any_distant_mets_years', 'local_failure', 'local_failure_years', 'biochemical_failure', 'biochemical_failure_years'])

In [ ]:
split_percentage = 0.5
scaler_type = "minmax"
feature_Sel_type = "ElasticNet"
cohort = "RTOG_0521_OS_spatil_plus_nucdiv"
l1_ratio = 0.9
risk_treshold_method = "median"
num_features = 5
N_runs = 50
seed = 0
random.seed(seed)

In [ ]:
imputer = SimpleImputer(strategy='median')
rtog_features = features.drop(['patient_id', 'RX', 'time', 'event'], axis=1) # 
rtog_features.replace([np.inf, -np.inf], np.nan, inplace=True)
num_df = rtog_features.values
names = rtog_features.columns.values
rtog_features = pd.DataFrame(imputer.fit_transform(num_df), columns=names)
rtog_features['patient_id'] = features['patient_id']
rtog_features['RX'] = features['RX']
rtog_features['time'] = features['time']
rtog_features['event'] = features['event']

##### Select one treatment arm to train

In [ ]:
rtog_leg_1 = rtog_features.loc[rtog_features['RX'] == 1]

In [ ]:
# select relevant features from both sets
selected_cols = ['patient_id', 'RX', 'time', 'event', 'nucdiv_60', 'spatil_0', 'spatil_341', 'nucdiv_293', 'spatil_51', 'spatil_336', 'nucdiv_360']
selected_df = rtog_leg_1[selected_cols]
training_data, holdout_data = train_test_split(selected_df, test_size=split_percentage, random_state=seed, stratify=selected_df['event'])


In [ ]:
training_data

In [ ]:
scaler = MinMaxScaler()
X_train = training_data.drop(columns=['patient_id', 'event', 'time', 'RX'], axis=1)
X_val = holdout_data.drop(columns=['patient_id', 'event', 'time', 'RX'], axis=1)
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)
X_val = pd.DataFrame(scaler.transform(X_val), columns=X_val.columns)
X_train['patient_id'] = training_data['patient_id'].values
X_train['time'] = training_data['time'].values
X_train['event'] = training_data['event'].values
X_val['patient_id'] = holdout_data['patient_id'].values
X_val['time'] = holdout_data['time'].values
X_val['event'] = holdout_data['event'].values

In [ ]:
y_training_survlabel = Surv.from_dataframe('event', 'time', training_data)
y_holdout_survlabel = Surv.from_dataframe('event', 'time', holdout_data)

In [ ]:
X_training = X_train.drop(['patient_id', 'event', 'time'], axis=1)
X_holdout = X_val.drop(['patient_id', 'event', 'time'], axis=1)
y_training = X_train[['patient_id','event', 'time']].reset_index(drop=True)
y_holdout = X_val[['patient_id','event', 'time']].reset_index(drop=True)

In [ ]:
cph = CoxPHFitter()
cph.fit(X_train.drop(columns=['patient_id']), duration_col='time', event_col='event', show_progress=False)
cph.print_summary()

In [ ]:
training_risk_scores = cph.predict_partial_hazard(X_train)
holdout_risk_scores = cph.predict_partial_hazard(X_holdout)

In [ ]:
risk_threshold = np.median(training_risk_scores)

In [ ]:
risk_threshold

In [ ]:
events, times = y_training['event'].values.astype(bool),   y_training['time'].values

In [ ]:
train_risk_group_labels = np.array([1 if x > risk_threshold else 0 for x in training_risk_scores])#np.digitize(train_risk_scores, risk_groups)
train_survival_probs = []
train_survival_times = []
events, times = y_training['event'].values.astype(bool),   y_training['time'].values

for group_label in np.unique(train_risk_group_labels):
    group_indices = np.where(train_risk_group_labels == group_label)
    group_time, group_survival_prob = kaplan_meier_estimator(events[group_indices], times[group_indices])
    train_survival_probs.append(group_survival_prob)
    train_survival_times.append(group_time)

events, times = y_holdout['event'].values.astype(bool),   y_holdout['time'].values
holdout_risk_group_labels = np.array([1 if x > risk_threshold else 0 for x in holdout_risk_scores])#np.digitize(test_risk_scores, risk_groups)
holdout_survival_probs = []
holdout_survival_times = []

for group_label in np.unique(holdout_risk_group_labels):
    group_indices = np.where(holdout_risk_group_labels == group_label)
    group_time, group_survival_prob = kaplan_meier_estimator(events[group_indices], times[group_indices])
    holdout_survival_probs.append(group_survival_prob)
    holdout_survival_times.append(group_time)

In [ ]:
y_holdout['risk_score'] = holdout_risk_scores
y_holdout['risk_group'] = holdout_risk_group_labels
y_holdout['patient_id'] = y_holdout['patient_id']
y_holdout = y_holdout.sort_values(by=['patient_id']).reset_index(drop=True)
y_holdout

In [ ]:
y_training['risk_score'] = training_risk_scores
y_training['risk_group'] = train_risk_group_labels
y_training['patient_id'] = y_training['patient_id']
y_training = y_training.sort_values(by=['patient_id']).reset_index(drop=True)
y_training

In [ ]:
LL_train = y_training.drop(['patient_id', 'risk_score'], axis=1)
LL_holdout = y_holdout.drop(['patient_id', 'risk_score'], axis=1)
cph_train = CoxPHFitter()
cph_train.fit(LL_train, duration_col='time', event_col='event', show_progress=False)
cph_holdout = CoxPHFitter()
cph_holdout.fit(LL_holdout, duration_col='time', event_col='event', show_progress=False)
training_results = cph_train.summary
training_p = multivariate_logrank_test(y_training['time'], y_training['risk_group'], y_training['event']).p_value# training_results['p'].values[0]
training_hr = training_results['exp(coef)'].values[0]
training_ci_lower = training_results['exp(coef) lower 95%'].values[0]
training_ci_upper = training_results['exp(coef) upper 95%'].values[0]
training_log_likelihood = cph_train.log_likelihood_
model_metrics = pd.DataFrame()
model_metrics['training_p_value'] = [training_p]
model_metrics['training_hazard_ratio'] = [training_hr]
model_metrics['training_hr_ci_lower'] = [training_ci_lower]
model_metrics['training_hr_ci_upper'] = [training_ci_upper]
model_metrics['training_log_likelihood'] = [training_log_likelihood]
model_metrics['training_parameters'] = [cph_train.params_.shape[0]]
training_data_stats = (training_p, training_hr, training_ci_lower, training_ci_upper)
holdout_results = cph_holdout.summary
holdout_p = multivariate_logrank_test(y_holdout['time'], y_holdout['risk_group'], y_holdout['event']).p_value # holdout_results['p'].values[0]
holdout_hr = holdout_results['exp(coef)'].values[0]
holdout_ci_lower = holdout_results['exp(coef) lower 95%'].values[0]
holdout_ci_upper = holdout_results['exp(coef) upper 95%'].values[0]
holdout_log_likelihood = cph_holdout.log_likelihood_
model_metrics['holdout_p_value'] = [holdout_p]
model_metrics['holdout_hazard_ratio'] = [holdout_hr]
model_metrics['holdout_hr_ci_lower'] = [holdout_ci_lower]
model_metrics['holdout_hr_ci_upper'] = [holdout_ci_upper]
model_metrics['holdout_log_likelihood'] = [holdout_log_likelihood]
model_metrics['holdout_parameters'] = [cph_holdout.params_.shape[0]]
holdout_data_stats = (holdout_p, holdout_hr, holdout_ci_lower, holdout_ci_upper)

In [ ]:
cph_train.print_summary()

In [ ]:
cph_holdout.print_summary()

In [ ]:
train_title_str = 'Training Set - RT+ADT treated patients (N={:d})'.format(len(y_training))
train_figure_save_path = f'KM_curve_training.png'
plot_km_curve_lifelines(y_training, training_data_stats, train_figure_save_path, train_title_str)

In [ ]:
holdout_title_str = 'Holdout Set - RT+ADT treated patients (N={:d})'.format(len(y_holdout))
holdout_figure_save_path = f'KM_curve_holdout.png'
plot_km_curve_lifelines(y_holdout, holdout_data_stats, holdout_figure_save_path, holdout_title_str)

### LEG 2

In [ ]:
rtog_leg_2 = rtog_features.loc[rtog_features['RX'] == 2]
rtog_leg_2 = rtog_leg_2[selected_cols]
X_holdout_chemo_arm = rtog_leg_2.drop(['patient_id', 'event', 'time', 'RX'], axis=1).reset_index(drop=True)
y_holdout_chemo_arm = rtog_leg_2[['patient_id','event', 'time', 'RX']].reset_index(drop=True)
X_holdout_chemo_arm = pd.DataFrame(scaler.transform(X_holdout_chemo_arm), columns=X_holdout_chemo_arm.columns)
y_holdout_chemo_arm_survlabel = Surv.from_dataframe('event', 'time', y_holdout_chemo_arm)

In [ ]:
chemo_arm_risk_scores = cph.predict_partial_hazard(X_holdout_chemo_arm)

In [ ]:
chemo_arm_events, chemo_arm_times = y_holdout_chemo_arm['event'].values.astype(bool), y_holdout_chemo_arm['time'].values
holdout_chemo_arm_risk_group_labels = np.array([1 if x > risk_threshold else 0 for x in chemo_arm_risk_scores])
holdout_chemo_arm_survival_probs = []
holdout_chemo_arm_survival_times = []
for group_label in np.unique(holdout_chemo_arm_risk_group_labels):
    group_indices = np.where(holdout_chemo_arm_risk_group_labels == group_label)
    group_time, group_survival_prob = kaplan_meier_estimator(chemo_arm_events[group_indices], chemo_arm_times[group_indices])
    holdout_survival_probs.append(group_survival_prob)
    holdout_survival_times.append(group_time)

In [ ]:
y_holdout_chemo_arm['risk_score'] = chemo_arm_risk_scores
y_holdout_chemo_arm['risk_group'] = holdout_chemo_arm_risk_group_labels
y_holdout_chemo_arm['patient_id'] = y_holdout_chemo_arm['patient_id']
y_holdout_chemo_arm = y_holdout_chemo_arm.sort_values(by=['patient_id']).reset_index(drop=True)
y_holdout_chemo_arm

In [ ]:
y_holdout_chemo_arm

In [ ]:
LL_holdout_chemo_arm = y_holdout_chemo_arm.drop(['patient_id', 'risk_score', 'RX'], axis=1)
cph_holdout_chemo_arm = CoxPHFitter()
cph_holdout_chemo_arm.fit(LL_holdout_chemo_arm, duration_col='time', event_col='event', show_progress=False)
chemo_arm_holdout_results = cph_holdout_chemo_arm.summary
chemo_arm_holdout_p = multivariate_logrank_test(y_holdout_chemo_arm['time'], y_holdout_chemo_arm['risk_group'], y_holdout_chemo_arm['event']).p_value # holdout_results['p'].values[0]
chemo_arm_holdout_hr = chemo_arm_holdout_results['exp(coef)'].values[0]
chemo_arm_holdout_ci_lower = chemo_arm_holdout_results['exp(coef) lower 95%'].values[0]
chemo_arm_holdout_ci_upper = chemo_arm_holdout_results['exp(coef) upper 95%'].values[0]
chemo_arm_holdout_log_likelihood = cph_holdout_chemo_arm.log_likelihood_
model_metrics_chemo_arm = pd.DataFrame()
model_metrics_chemo_arm['holdout_p_value'] = [chemo_arm_holdout_p]
model_metrics_chemo_arm['holdout_hazard_ratio'] = [chemo_arm_holdout_hr]
model_metrics_chemo_arm['holdout_hr_ci_lower'] = [chemo_arm_holdout_ci_lower]
model_metrics_chemo_arm['holdout_hr_ci_upper'] = [chemo_arm_holdout_ci_upper]
model_metrics_chemo_arm['holdout_log_likelihood'] = [chemo_arm_holdout_log_likelihood]
model_metrics_chemo_arm['holdout_parameters'] = [cph_holdout_chemo_arm.params_.shape[0]]
chemo_arm_holdout_data_stats = (chemo_arm_holdout_p, chemo_arm_holdout_hr, chemo_arm_holdout_ci_lower, chemo_arm_holdout_ci_upper)

In [ ]:
cph_holdout_chemo_arm.print_summary()

In [ ]:
chemo_arm_holdout_title_str = 'Test Set - RT+ADT+CT (Docetaxel) treated patients(N={:d})'.format(len(y_holdout_chemo_arm))
chemo_arm_holdout_figure_save_path = f'KM_curve_holdout_chemo_arm.png'
plot_km_curve_lifelines(y_holdout_chemo_arm, chemo_arm_holdout_data_stats, chemo_arm_holdout_figure_save_path, chemo_arm_holdout_title_str)

#### APIC-low RT+ADT vs RT+ADT+CT

In [ ]:
chemo_favorable_risk = y_holdout_chemo_arm[y_holdout_chemo_arm['risk_group'] == 0]
chemo_favorable_risk['chemo'] = 1
no_chemo_favorable_risk = y_holdout[y_holdout['risk_group'] == 0]
no_chemo_favorable_risk['chemo'] = 0
favorable_risk = pd.concat([chemo_favorable_risk, no_chemo_favorable_risk])

In [ ]:
LL_holdout_chemo_arm = favorable_risk.drop(['patient_id', 'risk_group', 'risk_score', 'RX'], axis=1)
cph_holdout_chemo_arm = CoxPHFitter()
cph_holdout_chemo_arm.fit(LL_holdout_chemo_arm, duration_col='time', event_col='event', show_progress=False)

chemo_arm_holdout_results = cph_holdout_chemo_arm.summary
chemo_arm_holdout_p = multivariate_logrank_test(favorable_risk['time'], favorable_risk['chemo'], favorable_risk['event']).p_value # holdout_results['p'].values[0] # change group to chemo/nochemo
chemo_arm_holdout_hr = chemo_arm_holdout_results['exp(coef)'].values[0]
chemo_arm_holdout_ci_lower = chemo_arm_holdout_results['exp(coef) lower 95%'].values[0]
chemo_arm_holdout_ci_upper = chemo_arm_holdout_results['exp(coef) upper 95%'].values[0]
chemo_arm_holdout_log_likelihood = cph_holdout_chemo_arm.log_likelihood_
model_metrics_chemo_arm['holdout_p_value'] = [chemo_arm_holdout_p]
model_metrics_chemo_arm['holdout_hazard_ratio'] = [chemo_arm_holdout_hr]
model_metrics_chemo_arm['holdout_hr_ci_lower'] = [chemo_arm_holdout_ci_lower]
model_metrics_chemo_arm['holdout_hr_ci_upper'] = [chemo_arm_holdout_ci_upper]
model_metrics_chemo_arm['holdout_log_likelihood'] = [chemo_arm_holdout_log_likelihood]
model_metrics_chemo_arm['holdout_parameters'] = [cph_holdout_chemo_arm.params_.shape[0]]

chemo_arm_holdout_data_stats = (chemo_arm_holdout_p, chemo_arm_holdout_hr, chemo_arm_holdout_ci_lower, chemo_arm_holdout_ci_upper)

In [ ]:
plot_km_same_risk_group(no_chemo_favorable_risk, chemo_favorable_risk, chemo_arm_holdout_data_stats, 'APIC_low_both_legs_km.png', "APIC-low group RT+ADT vs RT+ADT+CT")

#### APIC-high RT+ADT vs RT+ADT+CT

In [ ]:
chemo_bad_risk = y_holdout_chemo_arm[y_holdout_chemo_arm['risk_group'] == 1]
no_chemo_bad_risk = y_holdout[y_holdout['risk_group'] == 1]
chemo_bad_risk['chemo'] = 1
no_chemo_bad_risk['chemo'] = 0
bad_risk = pd.concat([chemo_bad_risk, no_chemo_bad_risk])

In [ ]:
LL_holdout_chemo_arm = bad_risk.drop(['patient_id', 'risk_group', 'risk_score', 'RX'], axis=1)
cph_holdout_chemo_arm = CoxPHFitter()
cph_holdout_chemo_arm.fit(LL_holdout_chemo_arm, duration_col='time', event_col='event', show_progress=False)

chemo_arm_holdout_results = cph_holdout_chemo_arm.summary
chemo_arm_holdout_p = multivariate_logrank_test(bad_risk['time'], bad_risk['chemo'], bad_risk['event']).p_value # holdout_results['p'].values[0] # change group to chemo/nochemo
chemo_arm_holdout_hr = chemo_arm_holdout_results['exp(coef)'].values[0]
chemo_arm_holdout_ci_lower = chemo_arm_holdout_results['exp(coef) lower 95%'].values[0]
chemo_arm_holdout_ci_upper = chemo_arm_holdout_results['exp(coef) upper 95%'].values[0]
chemo_arm_holdout_log_likelihood = cph_holdout_chemo_arm.log_likelihood_
model_metrics_chemo_arm['holdout_p_value'] = [chemo_arm_holdout_p]
model_metrics_chemo_arm['holdout_hazard_ratio'] = [chemo_arm_holdout_hr]
model_metrics_chemo_arm['holdout_hr_ci_lower'] = [chemo_arm_holdout_ci_lower]
model_metrics_chemo_arm['holdout_hr_ci_upper'] = [chemo_arm_holdout_ci_upper]
model_metrics_chemo_arm['holdout_log_likelihood'] = [chemo_arm_holdout_log_likelihood]
model_metrics_chemo_arm['holdout_parameters'] = [cph_holdout_chemo_arm.params_.shape[0]]

chemo_arm_holdout_data_stats = (chemo_arm_holdout_p, chemo_arm_holdout_hr, chemo_arm_holdout_ci_lower, chemo_arm_holdout_ci_upper)

In [ ]:
plot_km_same_risk_group(no_chemo_bad_risk, chemo_bad_risk, chemo_arm_holdout_data_stats, 'APIC_high_both_legs_km.png', "APIC High RT+ADT vs RT+ADT+CT spaTIL+NucDiv")

#### Interaction term analysis

In [ ]:
interaction_term_test_df = pd.concat([no_chemo_bad_risk, chemo_bad_risk, no_chemo_favorable_risk, chemo_favorable_risk])
interaction_term_test_df['interaction_term'] = interaction_term_test_df['chemo'] * interaction_term_test_df['risk_score']
interaction_term_cox = CoxPHFitter()
interaction_term_cox.fit(interaction_term_test_df.drop(['patient_id', 'risk_group', 'RX'], axis=1), duration_col='time', event_col='event', show_progress=False)
interaction_term_cox_results = interaction_term_cox.summary

#### Net benefit analysis

In [ ]:
benefits = calculate_survival_benefit(no_chemo_favorable_risk, chemo_favorable_risk)
print("Survival Benefits at specified time points for biomarker - patients:", benefits)

In [ ]:
benefits = calculate_survival_benefit(no_chemo_bad_risk, chemo_bad_risk)
print("Survival Benefits at specified time points for biomarker + patients:", benefits)

In [ ]:
net_benefit_df = interaction_term_test_df[['event', 'time', 'risk_group']]

In [ ]:
net_benefit_df

In [ ]:
cox_dca = CoxPHFitter()
cox_dca.fit(net_benefit_df, duration_col='time', event_col='event', show_progress=False)

In [ ]:
cox_dca.summary

In [ ]:
cph_pred_vals = cox_dca.predict_survival_function(net_benefit_df[['risk_group']], times=[10])

In [ ]:
net_benefit_df['Predictive model'] = [1 - val for val in cph_pred_vals.iloc[0,:]]

In [ ]:
stdca_coxph_results = dca(data=net_benefit_df, outcome='event', modelnames=['Predictive model'], thresholds=np.arange(0,0.51,0.01), time=10, time_to_outcome_col='time')

plot_graphs(plot_df=stdca_coxph_results, graph_type='net_benefit', y_limits=[-0.05, 0.25], color_names=['green', 'red', 'blue'], show_grid=False, dpi=300)

In [ ]:
net_benefit_df_no_chemo = interaction_term_test_df[interaction_term_test_df['chemo'] == 0]

In [ ]:
net_benefit_df_no_chemo = net_benefit_df_no_chemo[['event', 'time', 'risk_group']]

In [ ]:
cox_dca = CoxPHFitter()
cox_dca.fit(net_benefit_df_no_chemo, duration_col='time', event_col='event', show_progress=False)

In [ ]:
cph_pred_vals = cox_dca.predict_survival_function(net_benefit_df_no_chemo[['risk_group']], times=[10])

In [ ]:
net_benefit_df_no_chemo['Predictive model'] = [1 - val for val in cph_pred_vals.iloc[0,:]]

In [ ]:
stdca_coxph_results = dca(data=net_benefit_df_no_chemo, outcome='event', modelnames=['Predictive model'], thresholds=np.arange(0,0.51,0.01), time=10, time_to_outcome_col='time')

plot_graphs(plot_df=stdca_coxph_results, graph_type='net_benefit', y_limits=[-0.05, 0.25], color_names=['green', 'red', 'blue'], show_grid=False, dpi=300)

In [ ]:
net_benefit_df_chemo = interaction_term_test_df[interaction_term_test_df['chemo'] == 1]
net_benefit_df_chemo = net_benefit_df_chemo[['event', 'time', 'risk_group']]
cox_dca = CoxPHFitter()
cox_dca.fit(net_benefit_df_chemo, duration_col='time', event_col='event', show_progress=False)
cph_pred_vals = cox_dca.predict_survival_function(net_benefit_df_chemo[['risk_group']], times=[10])
net_benefit_df_chemo['Predictive model'] = [1 - val for val in cph_pred_vals.iloc[0,:]]
stdca_coxph_results = dca(data=net_benefit_df_chemo, outcome='event', modelnames=['Predictive model'], thresholds=np.arange(0,0.51,0.01), time=10, time_to_outcome_col='time')
plot_graphs(plot_df=stdca_coxph_results, graph_type='net_benefit', y_limits=[-0.05, 0.25], color_names=['green', 'red', 'blue'], show_grid=False, dpi=300)

#### Multivariable analysis

In [ ]:
clinical_data = pd.read_csv('clinical_data.csv')

In [ ]:
clinical_data = clinical_data.rename(columns={'cn_deidentified': 'patient_id'})
clinical_data = clinical_data[['patient_id', 'age', 'race', 'gleason', 'psa', 't_stage']]
multivar_analysis_df = interaction_term_test_df.merge(clinical_data, on='patient_id', how='left')

In [ ]:
multivar_analysis_df.drop(columns=['risk_score', 'RX', 'interaction_term'], inplace=True)

In [ ]:
multivar_analysis_all_patients = multivar_analysis_df.drop(columns=['chemo'], axis=1)

In [ ]:
multivar_analysis_all_patients

In [ ]:
multivar_analysis_all_patients.drop(columns=['age', 'race'], inplace=True)
multivar_analysis_all_patients['t_stage'] = np.where(multivar_analysis_all_patients['t_stage'] > 7, 1, 0)

In [ ]:
holdout_results, cph_holdout_adjusted_risk = get_multivariate_coxph_results(multivar_analysis_all_patients)
plt_title = 'Risk Model controlling for clinical variables'
save_path = ''
plot_n_save_adjusted_hazard(cph_holdout_adjusted_risk, cph_holdout_adjusted_risk, multivar_analysis_all_patients, multivar_analysis_all_patients, save_path, plt_title)
cph_holdout_adjusted_risk.print_summary()

In [ ]:
multivar_analysis_df_only_chemo = multivar_analysis_df[multivar_analysis_df['chemo'] == 1]

In [ ]:
multivar_analysis_df_only_chemo

In [ ]:
multivar_analysis_df_only_chemo.drop(columns=['age', 'race', 'chemo'], inplace=True)
multivar_analysis_df_only_chemo['t_stage'] = np.where(multivar_analysis_df_only_chemo['t_stage'] > 7, 1, 0)

In [ ]:
holdout_results, cph_holdout_adjusted_risk = get_multivariate_coxph_results(multivar_analysis_df_only_chemo)
plt_title = 'Risk Model controlling for clinical variables (CT arm)'
save_path = ''
plot_n_save_adjusted_hazard(cph_holdout_adjusted_risk, cph_holdout_adjusted_risk, multivar_analysis_df_only_chemo, multivar_analysis_df_only_chemo, save_path, plt_title)
cph_holdout_adjusted_risk.print_summary()

In [ ]:
multivar_analysis_df_no_chemo = multivar_analysis_df[multivar_analysis_df['chemo'] == 0]
multivar_analysis_df_no_chemo.drop(columns=['age', 'race', 'chemo'], inplace=True)
multivar_analysis_df_no_chemo['t_stage'] = np.where(multivar_analysis_df_no_chemo['t_stage'] > 7, 1, 0)

In [ ]:
holdout_results, cph_holdout_adjusted_risk = get_multivariate_coxph_results(multivar_analysis_df_no_chemo)
plt_title = 'Risk Model controlling for clinical variables (RT+ADT arm)'
save_path = ''
plot_n_save_adjusted_hazard(cph_holdout_adjusted_risk, cph_holdout_adjusted_risk, multivar_analysis_df_no_chemo, multivar_analysis_df_no_chemo, save_path, plt_title)
cph_holdout_adjusted_risk.print_summary()

In [ ]:
results = {}
variables = ['risk_group','psa', 't_stage', 'gleason']
cph = CoxPHFitter()
for var in variables:

    cph.fit(multivar_analysis_all_patients[[var, 'time', 'event']], duration_col='time', event_col='event')
    
    # Extract the HR and its confidence intervals for the variable
    cph_model_results = cph.summary
    p = cph_model_results['p'].values[0]
    hr = cph_model_results['exp(coef)'].values[0]
    ci_lower = cph_model_results['exp(coef) lower 95%'].values[0]
    ci_upper = cph_model_results['exp(coef) upper 95%'].values[0]
    # Store the results
    results[var] = {'HR': hr, '95% CI Lower': ci_lower, '95% CI Upper': ci_upper, 'p': p}

# Display the results
print("Univariate analysis all patients")
for var, stats in results.items():
    print(f"{var}: HR = {stats['HR']:.4f}, 95% CI = ({stats['95% CI Lower']:.4f}, {stats['95% CI Upper']:.4f}), p = {stats['p']:.6f}")

In [ ]:
results = {}
variables = ['risk_group','psa', 't_stage', 'gleason']
cph = CoxPHFitter()
for var in variables:

    cph.fit(multivar_analysis_df_only_chemo[[var, 'time', 'event']], duration_col='time', event_col='event')
    
    # Extract the HR and its confidence intervals for the variable
    cph_model_results = cph.summary
    p = cph_model_results['p'].values[0]
    hr = cph_model_results['exp(coef)'].values[0]
    ci_lower = cph_model_results['exp(coef) lower 95%'].values[0]
    ci_upper = cph_model_results['exp(coef) upper 95%'].values[0]
    # Store the results
    results[var] = {'HR': hr, '95% CI Lower': ci_lower, '95% CI Upper': ci_upper, 'p': p}

# Display the results
print("Univariate analysis CT patients")
for var, stats in results.items():
    print(f"{var}: HR = {stats['HR']:.4f}, 95% CI = ({stats['95% CI Lower']:.4f}, {stats['95% CI Upper']:.4f}), p = {stats['p']:.6f}")

In [ ]:
results = {}
variables = ['risk_group','psa', 't_stage', 'gleason']
cph = CoxPHFitter()
for var in variables:

    cph.fit(multivar_analysis_df_no_chemo[[var, 'time', 'event']], duration_col='time', event_col='event')
    
    # Extract the HR and its confidence intervals for the variable
    cph_model_results = cph.summary
    p = cph_model_results['p'].values[0]
    hr = cph_model_results['exp(coef)'].values[0]
    ci_lower = cph_model_results['exp(coef) lower 95%'].values[0]
    ci_upper = cph_model_results['exp(coef) upper 95%'].values[0]
    # Store the results
    results[var] = {'HR': hr, '95% CI Lower': ci_lower, '95% CI Upper': ci_upper, 'p': p}

# Display the results
print("Univariate analysis no CT patients")
for var, stats in results.items():
    print(f"{var}: HR = {stats['HR']:.4f}, 95% CI = ({stats['95% CI Lower']:.4f}, {stats['95% CI Upper']:.4f}), p = {stats['p']:.6f}")